In [104]:
from sklearn import *
from tensorflow import keras as keras
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.datasets import load_digits
from tensorflow.keras.utils import to_categorical
import pandas as pd
import os
root_dir = r'/Users/pankaj/Library/Mobile Documents/com~apple~CloudDocs/Capstone/Wikipedia Data'


result_path = annot_file_path = os.path.join(root_dir , 'comments_with_grouped_annoptations.tsv')

merged_comments = pd.read_table(result_path)


In [105]:
print (merged_comments.head())

   Unnamed: 0  rev_id                                            comment  \
0           0   37675  `-NEWLINE_TOKENThis is not ``creative``.  Thos...   
1           1   44816  `NEWLINE_TOKENNEWLINE_TOKEN:: the term ``stand...   
2           2   49851  NEWLINE_TOKENNEWLINE_TOKENTrue or false, the s...   
3           3   89320   Next, maybe you could work on being less cond...   
4           4   93890               This page will need disambiguation.    

   year  logged_in       ns  sample  split  worker_id  quoting_attack  \
0  2002      False  article  random  train       9721             0.0   
1  2002      False  article  random  train      20234             0.0   
2  2002      False  article  random  train      26474             0.0   
3  2002       True  article  random    dev      26738             0.0   
4  2002       True  article  random  train       8010             0.0   

   recipient_attack  third_party_attack  other_attack  attack  
0               0.0                 0.0 

In [106]:
one_hot_train_labels

0         0
1         0
2         0
3         0
4         0
         ..
115859    0
115860    0
115861    0
115862    0
115863    0
Name: recipient_attack, Length: 115864, dtype: int64

In [107]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np



In [109]:
merged_comments['recipient_attack'] = merged_comments['recipient_attack'].apply(lambda x : 1 if x> 1 else 0 )
X_train = merged_comments['comment']
y = merged_comments['recipient_attack']
one_hot_train_labels = y

In [110]:
one_hot_train_labels.shape

(115864,)

In [111]:
one_hot_train_labels

0         0
1         0
2         0
3         1
4         0
         ..
115859    0
115860    0
115861    0
115862    0
115863    0
Name: recipient_attack, Length: 115864, dtype: int64

In [112]:
merged_comments['recipient_attack'].unique()

array([0, 1])

In [122]:
### TODO change model with proper word settings 

In [113]:
# Prepare training input  
maxlen = 100
training_samples = 20000
validation_samples = 10000
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=maxlen)
print('Shape of data tensor:', data.shape)
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
x_train = data[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]

Found 187899 unique tokens.
Shape of data tensor: (115864, 100)


In [114]:
# Prepare labels 
labels = np.asarray(one_hot_train_labels)
print('Shape of label tensor:', labels.shape)
labels = labels[indices]
y_train = labels[:training_samples]
y_val = labels[training_samples: training_samples + validation_samples]

Shape of label tensor: (115864,)


In [115]:
tb_collback = keras.callbacks.TensorBoard(log_dir = 'logs_classification', histogram_freq= 1)
early_sp_cb= keras.callbacks.EarlyStopping(monitor='acc', min_delta=0.01, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
callbacks = [tb_collback, early_sp_cb]

In [116]:
from tensorflow.keras.layers import LSTM
embedding_dim = 100
model = Sequential()
embedding = Embedding(max_words, embedding_dim, input_length=maxlen)
embedding.trainable = False
model.add(embedding)
model.add(LSTM(32))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
lstm_9 (LSTM)                (None, 32)                17024     
_________________________________________________________________
flatten_9 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 33        
Total params: 1,018,113
Trainable params: 18,113
Non-trainable params: 1,000,000
_________________________________________________________________


In [117]:
y_train

array([0, 0, 0, ..., 0, 0, 1])

In [118]:
y_train.shape

(20000,)

In [119]:
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['acc'])
history = model.fit(x_train, y_train,
epochs=2,
batch_size=256,
validation_data=(x_val, y_val), callbacks= callbacks)

Train on 20000 samples, validate on 10000 samples
Epoch 1/2
20000/20000 [==============================] - 11s 531us/sample - loss: 0.5493 - acc: 0.7673 - val_loss: 0.5414 - val_acc: 0.7630
Epoch 2/2
20000/20000 [==============================] - 9s 447us/sample - loss: 0.5137 - acc: 0.7707 - val_loss: 0.5079 - val_acc: 0.7641


In [123]:
#predictions = model.predict(test_data)

In [121]:
import os
PATH = os.getenv('PATH')
%env PATH=/Users/pankaj/anaconda/envs/nlp/bin:$PATH
    
%load_ext tensorboard
%tensorboard --logdir logs_classification


env: PATH=/Users/pankaj/anaconda/envs/nlp/bin:/Users/pankaj/anaconda/condabin:/Users/pankaj/anaconda/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/opt/X11/bin
